# YOLO CAR
2023 08 16  
1540i  
let get init  

In [9]:
import os
current_dir = os.getcwd()  #Here
# List the contents of the "data" directory
train_path = os.path.join(current_dir, "training_images")
data_directory_contents = os.listdir(train_path)
data_directory_contents[:5]

['vid_4_18340.jpg',
 'vid_4_2460.jpg',
 'vid_4_10720.jpg',
 'vid_4_14220.jpg',
 'vid_4_6160.jpg']

In [10]:
import os
current_dir = os.getcwd()  #Here
# List the contents of the "data" directory
test_path = os.path.join(current_dir, "testing_images")
data_directory_contents = os.listdir(test_path)
data_directory_contents[:5]

['vid_5_30040.jpg',
 'vid_5_27500.jpg',
 'vid_5_28380.jpg',
 'vid_5_26420.jpg',
 'vid_5_31160.jpg']

In [12]:
import pandas as pd

# Load the bounding boxes CSV file
bounding_boxes_csv_path = os.path.join(current_dir, "train_solution_bounding_boxes.csv")
bounding_boxes_df = pd.read_csv(bounding_boxes_csv_path)

# Display the first few rows of the dataframe
bounding_boxes_df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


In [14]:
from PIL import Image

def convert_to_yolo_format(row, img_path):
    # Open the image to get its width and height
    with Image.open(img_path) as img:
        img_width, img_height = img.size
    
    # Calculate center, width and height of the bounding box
    center_x = (row['xmin'] + row['xmax']) / 2.0
    center_y = (row['ymin'] + row['ymax']) / 2.0
    width = row['xmax'] - row['xmin']
    height = row['ymax'] - row['ymin']
    
    # Normalize the values
    center_x /= img_width
    center_y /= img_height
    width /= img_width
    height /= img_height
    
    return f"0 {center_x} {center_y} {width} {height}"

# Apply the conversion to each row in the dataframe
training_images_path = os.path.join(current_dir, "training_images")
bounding_boxes_df['yolo_format'] = bounding_boxes_df.apply(lambda row: convert_to_yolo_format(row, os.path.join(training_images_path, row['image'])), axis=1)

# Display the first few rows of the updated dataframe
bounding_boxes_df.head()

,image,xmin,ymin,xmax,ymax,yolo_format
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547,0 0.45043415340236687 0.539816602368421 0.0687...
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180,0 0.10021707670857989 0.5571911197368421 0.155...
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180,0 0.44464544142011836 0.5436776061842105 0.181...
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575,0 0.8332127352071006 0.5314510939473683 0.1975...
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422,0 0.11034732271449704 0.5591216215789474 0.171...


In [15]:
# Directory to save YOLO formatted labels
labels_directory = os.path.join(current_dir, "yolo_labels")
os.makedirs(labels_directory, exist_ok=True)

for _, row in bounding_boxes_df.iterrows():
    # Create a text file for each image with the corresponding YOLO format data
    txt_filename = os.path.splitext(row['image'])[0] + '.txt'
    txt_path = os.path.join(labels_directory, txt_filename)
    with open(txt_path, 'w') as f:
        f.write(row['yolo_format'] + "\n")

# Check the first few created files
created_files = os.listdir(labels_directory)[:5]
created_files


['vid_4_740.txt',
 'vid_4_6260.txt',
 'vid_4_13840.txt',
 'vid_4_15000.txt',
 'vid_4_21400.txt']

In [16]:
# Create a names file for class names
classes_names_path = os.path.join(current_dir, "classes.names")
with open(classes_names_path, 'w') as f:
    f.write("object\n")  # Only one class named "object"

# Create a data file with paths to datasets and class names
data_file_content = f"""
classes = 1
train = {os.path.join(current_dir, "train.txt")}
valid = {os.path.join(current_dir, "valid.txt")}
names = {classes_names_path}
backup = backup/
"""

data_file_path = os.path.join(current_dir, "dataset.data")
with open(data_file_path, 'w') as f:
    f.write(data_file_content)

data_file_path


'/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/dataset.data'

In [17]:
import random

# Get all training image paths
all_image_paths = [os.path.join(training_images_path, img_name) for img_name in bounding_boxes_df['image'].tolist()]

# Shuffle and split the dataset into training and validation sets
random.shuffle(all_image_paths)
split_point = int(0.9 * len(all_image_paths))
train_images = all_image_paths[:split_point]
valid_images = all_image_paths[split_point:]

# Save the paths to train.txt and valid.txt files
train_txt_path = os.path.join(current_dir, "train.txt")
valid_txt_path = os.path.join(current_dir, "valid.txt")

with open(train_txt_path, 'w') as f:
    f.write("\n".join(train_images))

with open(valid_txt_path, 'w') as f:
    f.write("\n".join(valid_images))

train_txt_path, valid_txt_path


('/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/train.txt',
 '/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/valid.txt')

# Image size fix

In [18]:
# Open the first image to get its dimensions
with Image.open(all_image_paths[0]) as img:
    img_width, img_height = img.size

img_width, img_height

(676, 380)

In [19]:
# Calculate the network width and height as multiples of 32
network_width = (img_width + 31) // 32 * 32
network_height = (img_height + 31) // 32 * 32

network_width, network_height

(704, 384)

In [20]:
# YOLOv3 configuration for training
yolov3_config = f"""
[net]
# Testing
batch=4
subdivisions=2
# Training
# batch=64
# subdivisions=16
width={network_width}
height={network_height}
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# ... (Many more layers)

[region]
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
bias_match=1
classes=1
coords=4
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
"""

# Save the configuration to a file
cfg_path = os.path.join(current_dir, "yolov3_custom.cfg")
with open(cfg_path, 'w') as f:
    f.write(yolov3_config)

cfg_path

'/Users/owo/HOUSE/@Code/@Project/ID_CarObjectDetection/yolov3_custom.cfg'

# Data processing

In [ ]:
import numpy as np
import torch
class YOLODataset(Dataset):
    def __init__(self, list_path, img_size=416):
        with open(list_path, 'r') as file:
            self.img_files = file.readlines()
        
        self.label_files = [path.replace('images', 'labels').replace('.png', '.txt').replace('.jpg', '.txt') for path in self.img_files]
        self.img_size = img_size

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, index):
        img_path = self.img_files[index % len(self.img_files)].rstrip()
        img = Image.open(img_path).convert('RGB')
        img = img.resize((self.img_size, self.img_size))
        img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
        img = torch.from_numpy(img).permute(2, 0, 1)  # Convert to (C, H, W) format

        label_path = self.label_files[index % len(self.img_files)].rstrip()
        labels = torch.zeros((0, 5), dtype=torch.float32)  # Default empty label
        if os.path.exists(label_path):
            labels_data = [list(map(float, line.split())) for line in open(label_path)]
            if labels_data:
                labels = torch.tensor(labels_data, dtype=torch.float32)

        return img, labels

# Example usage
dataset = YOLODataset(train_txt_path, img_size=704)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)
next(iter(dataloader))


In [ ]:
import torch
from torch.utils.data import DataLoader
from models import Darknet
from utils.datasets import ListDataset
from utils.utils import compute_loss

# Configuration
cfg = '/path/to/yolov3_custom.cfg'
data_config = '/path/to/dataset.data'
img_size = 704
epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model and optimizer
model = Darknet(cfg, img_size=img_size).to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters())

# Get dataloader
train_path = '/path/to/train.txt'
dataset = ListDataset(train_path, img_size=img_size)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=dataset.collate_fn)

# Training loop
for epoch in range(epochs):
    for batch_i, (imgs, targets) in enumerate(dataloader):
        # Send to GPU
        imgs = imgs.to(device)
        targets = targets.to(device)
        
        # Forward pass
        outputs = model(imgs)
        loss, loss_components = compute_loss(outputs, targets, model)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Print log
        print(f"Epoch {epoch}/{epochs}, Batch {batch_i}/{len(dataloader)}, Loss: {loss.item()}")

    # Save model weights periodically
    if epoch % 10 == 0:
        torch.save(model.state_dict(), f"weights_epoch_{epoch}.pth")
